## Bank Churn Prediction
### Given a Bank customer, build a neural network-based classifier that can determine whether they will leave or not.

In [1]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 2.0.0


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers


In [3]:
bank = pd.read_csv('bank.csv')

In [4]:
bank.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
bank.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
bank = bank.drop("RowNumber", axis = 1)

In [8]:
bank = bank.drop("CustomerId", axis = 1)

In [9]:
bank = bank.drop("Surname", axis = 1)

In [10]:
bank.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
bank = pd.get_dummies(bank)
bank = bank.astype('float32') 
bank

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619.0,42.0,2.0,0.000000,1.0,1.0,1.0,101348.882812,1.0,1.0,0.0,0.0,1.0,0.0
1,608.0,41.0,1.0,83807.859375,1.0,0.0,1.0,112542.578125,0.0,0.0,0.0,1.0,1.0,0.0
2,502.0,42.0,8.0,159660.796875,3.0,1.0,0.0,113931.570312,1.0,1.0,0.0,0.0,1.0,0.0
3,699.0,39.0,1.0,0.000000,2.0,0.0,0.0,93826.632812,0.0,1.0,0.0,0.0,1.0,0.0
4,850.0,43.0,2.0,125510.820312,1.0,1.0,1.0,79084.101562,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771.0,39.0,5.0,0.000000,2.0,1.0,0.0,96270.640625,0.0,1.0,0.0,0.0,0.0,1.0
9996,516.0,35.0,10.0,57369.609375,1.0,1.0,1.0,101699.773438,0.0,1.0,0.0,0.0,0.0,1.0
9997,709.0,36.0,7.0,0.000000,1.0,0.0,1.0,42085.578125,1.0,1.0,0.0,0.0,1.0,0.0
9998,772.0,42.0,3.0,75075.312500,2.0,1.0,0.0,92888.523438,1.0,0.0,1.0,0.0,0.0,1.0


In [13]:
X_data = bank.drop("Exited", axis = 1)

In [14]:
X_data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619.0,42.0,2.0,0.000000,1.0,1.0,1.0,101348.882812,1.0,0.0,0.0,1.0,0.0
1,608.0,41.0,1.0,83807.859375,1.0,0.0,1.0,112542.578125,0.0,0.0,1.0,1.0,0.0
2,502.0,42.0,8.0,159660.796875,3.0,1.0,0.0,113931.570312,1.0,0.0,0.0,1.0,0.0
3,699.0,39.0,1.0,0.000000,2.0,0.0,0.0,93826.632812,1.0,0.0,0.0,1.0,0.0
4,850.0,43.0,2.0,125510.820312,1.0,1.0,1.0,79084.101562,0.0,0.0,1.0,1.0,0.0


In [21]:
X_data.shape

(10000, 13)

In [18]:
y_data = bank['Exited']

In [19]:
y_data

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
9995    0.0
9996    0.0
9997    1.0
9998    1.0
9999    0.0
Name: Exited, Length: 10000, dtype: float32

In [22]:
y_data.shape

(10000,)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.25, random_state = 17)

In [24]:
X_train = preprocessing.normalize(X_train)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7500, 13)
(2500, 13)
(7500,)
(2500,)


### Keras model object

In [59]:
model = Sequential()


In [60]:
model.add(Dense(80, input_shape = (13,), activation = 'relu'))
model.add(Dense(40, activation = 'tanh'))
model.add(Dense(1, activation = 'sigmoid'))

### Model Compile

In [77]:
sgd = optimizers.Adam(lr = 0.01)

In [78]:
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=['accuracy'])

### Model Summary

In [79]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 80)                1120      
_________________________________________________________________
dense_7 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 41        
Total params: 4,401
Trainable params: 4,401
Non-trainable params: 0
_________________________________________________________________


### Training the Model

In [80]:
model.fit(X_train, y_train.values, batch_size = 500, epochs = 12, verbose = 1)

Train on 7500 samples
Epoch 1/12
7500/7500 [==============================] - 0s 45us/sample - loss: 0.4990 - accuracy: 0.7959
Epoch 2/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4987 - accuracy: 0.7959
Epoch 3/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4988 - accuracy: 0.7959
Epoch 4/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4987 - accuracy: 0.7959
Epoch 5/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4986 - accuracy: 0.7959
Epoch 6/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4988 - accuracy: 0.7959
Epoch 7/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4985 - accuracy: 0.7959
Epoch 8/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4984 - accuracy: 0.7959
Epoch 9/12
7500/7500 [==============================] - 0s 5us/sample - loss: 0.4983 - accuracy: 0.7959
Epoch 10/12
7500/7500 [==================

### Evaluation

In [81]:
X_test = preprocessing.normalize(X_test)

In [82]:
results = model.evaluate(X_test, y_test.values)

2500/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [83]:
print(model.metrics_names)
print(results)

['loss', 'accuracy']
[0.4960862266540527, 0.7976]


### Confusion Metrics

In [88]:
Y_pred_cls = model.predict_classes(X_test, batch_size=50, verbose=0)
print('Accuracy Model1 (Dropout): '+ str(model.evaluate(X_test,y_test.values)[1]))
print('Recall_score: ' + str(recall_score(y_test.values,Y_pred_cls)))
print('Precision_score: ' + str(precision_score(y_test.values, Y_pred_cls)))
print('F-score: ' + str(f1_score(y_test.values,Y_pred_cls)))
confusion_matrix(y_test.values, Y_pred_cls)

2500/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Accuracy Model1 (Dropout): 0.7976
Recall_score: 0.0
Precision_score: 0.0
F-score: 0.0


array([[1994,    0],
       [ 506,    0]])

In [98]:
model2 = Sequential()



In [107]:
model2.add(Dense(80, input_shape = (13,), activation = 'relu'))
model2.add(Dense(40, activation = 'tanh'))
model2.add(Dense(1, activation = 'sigmoid'))

In [108]:
sgd2 = optimizers.Adagrad(lr = 0.01)

In [109]:
model2.compile(optimizer = sgd2, loss = 'binary_crossentropy', metrics=['accuracy'])

In [110]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 80)                1120      
_________________________________________________________________
dense_7 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 41        
Total params: 4,401
Trainable params: 4,401
Non-trainable params: 0
_________________________________________________________________


In [111]:
model2.fit(X_train, y_train.values, batch_size = 500, epochs = 12, verbose = 1)

Train on 7500 samples
Epoch 1/12
7500/7500 [==============================] - 1s 77us/sample - loss: 0.6688 - accuracy: 0.5977
Epoch 2/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5647 - accuracy: 0.7959
Epoch 3/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5289 - accuracy: 0.7959
Epoch 4/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5151 - accuracy: 0.7959
Epoch 5/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5097 - accuracy: 0.7959
Epoch 6/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5075 - accuracy: 0.7959
Epoch 7/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5066 - accuracy: 0.7959
Epoch 8/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5063 - accuracy: 0.7959
Epoch 9/12
7500/7500 [==============================] - 0s 4us/sample - loss: 0.5062 - accuracy: 0.7959
Epoch 10/12
7500/7500 [==================

In [112]:
results2 = model2.evaluate(X_test, y_test.values)

2500/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [113]:
print(model2.metrics_names)
print(results2)

['loss', 'accuracy']
[0.5037342891693115, 0.7976]


In [114]:
Y_pred_cls2 = model2.predict_classes(X_test, batch_size=50, verbose=0)
print('Accuracy Model1 (Dropout): '+ str(model2.evaluate(X_test,y_test.values)[1]))
print('Recall_score: ' + str(recall_score(y_test.values,Y_pred_cls)))
print('Precision_score: ' + str(precision_score(y_test.values, Y_pred_cls)))
print('F-score: ' + str(f1_score(y_test.values,Y_pred_cls)))
confusion_matrix(y_test.values, Y_pred_cls2)

2500/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Accuracy Model1 (Dropout): 0.7976
Recall_score: 0.0
Precision_score: 0.0
F-score: 0.0


array([[1994,    0],
       [ 506,    0]])